In [28]:
import cv2
import numpy as np
import os, os.path

In [29]:
def face_skin_detect(path):
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #Nhận diện vùng khuôn mặt
    faceCascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(30, 30)
    )
    mask = np.zeros(image.shape, np.uint8)
    for (x, y, w, h) in faces:
        mask[y:y+h, x:x+w] = image[y:y+h, x:x+w]
        
    #Đổi màu qua HSV, lấy các màu trong khoảng vùng màu người
    converted = cv2.cvtColor(mask, cv2.COLOR_BGR2HSV)
    lower_skin_color = np.array([0, 50, 60], dtype = "uint8")
    upper_skin_color = np.array([13, 150, 255], dtype = "uint8")
    skinMask = cv2.inRange(converted, lower_skin_color, upper_skin_color)
    
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 9))
    skinMask = cv2.erode(skinMask, kernel, iterations = 3)
    skinMask = cv2.dilate(skinMask, kernel, iterations = 5)
    skinMask = cv2.GaussianBlur(skinMask, (3, 3), 0)
    skin = cv2.bitwise_and(image, image, mask = skinMask)
    cv2.imwrite('./output/face_skin_detected_' + path, skin)

In [30]:
for path in os.listdir('./'):
    ext = os.path.splitext(path)[1]
    if ext.lower() == ".jpg":
        face_skin_detect('' + path)